In [1]:
import keras
import numpy as np

/home/ubuntu/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/home/ubuntu/anaconda3/lib/python3.6/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #2
  (fname, cnt))
/home/ubuntu/anaconda3/lib/python3.6/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #3
  (fname, cnt))


In [3]:
path = keras.utils.get_file(
    '1342-0.txt',
    origin='https://www.gutenberg.org/files/1342/1342-0.txt')
text = open(path).read().lower()
print('Corpus length:', len(text))

729088/724725 [==============================] - 0s 0us/step
Corpus length: 704190


In [21]:
maxlen = 60
step = 3
sentences = []
next_chars = []

for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('Number of sequences:', len(sentences))


chars = sorted(list(set(text)))
print('Unique characters:', len(chars))
char_indices = dict((char, chars.index(char)) for char in chars)


print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

Number of sequences: 234710
Unique characters: 60
Vectorization...


In [22]:
from keras import layers

model = keras.models.Sequential()
model.add(layers.LSTM(128, input_shape=(maxlen, len(chars))))
model.add(layers.Dense(len(chars), activation='softmax'))

In [23]:
optimizer = keras.optimizers.RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

In [24]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [26]:
import random
import sys

for epoch in range(1, 10):
    print('epoch', epoch)
    # Fit the model for 1 epoch on the available training data
    model.fit(x, y,
              batch_size=512,
              epochs=1)

    # Select a text seed at random
    start_index = random.randint(0, len(text) - maxlen - 1)
    generated_text = text[start_index: start_index + maxlen]
    print('--- Generating with seed: "' + generated_text + '"')

    for temperature in [0.2, 0.5, 1.0, 1.2]:
        print('------ temperature:', temperature)
        sys.stdout.write(generated_text)

        # We generate 180 characters
        for i in range(180):
            sampled = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(generated_text):
                sampled[0, t, char_indices[char]] = 1.

            preds = model.predict(sampled, verbose=0)[0]
            next_index = sample(preds, temperature)
            next_char = chars[next_index]

            generated_text += next_char
            generated_text = generated_text[1:]

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

epoch 1
Epoch 1/1
234710/234710 [==============================] - 323s 1ms/step - loss: 1.8450
--- Generating with seed: "t length the parsonage was discernible. the garden sloping t"
------ temperature: 0.2
t length the parsonage was discernible. the garden sloping to the pertions to be to the projection to the pertured to the could not the perted to be the parting to the projection to here to the could to the pertoution to the pertioned to be
------ temperature: 0.5
o here to the could to the pertoution to the pertioned to betined to the contresioul there was to the good the prefectioused to able.





hered been it could not it dadies we the could to the supterine which all the reatied to the pertion 
------ temperature: 1.0
could to the supterine which all the reatied to the pertion theermoid!
“yout foetlecumined muth
evices the fordined white is ang
step. whice widt obendesting, and penharm it
ible the poormed. which dowed
my. booution; forgitplation on erise
------ temperature: 

/home/ubuntu/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:3: RuntimeWarning: divide by zero encountered in log
  app.launch_new_instance()


y wishlement?”

“and lydge. mr.
darcy yout _oune_ changers, who was you.”

“rosionsest to be deproborate comfurted anded much to be wich, anythandshonsed tekered 
------ temperature: 1.2
te comfurted anded much to be wich, anythandshonsed tekered gotavour.
then quithed diffured by yesuen matore, to chauge; with chupted to formation, “how
was nother erentagenge willidston upose child elizaneterpy grouge for which _ondy conte
epoch 6
Epoch 1/1
234710/234710 [==============================] - 321s 1ms/step - loss: 1.4004
--- Generating with seed: "ond
of me; and i have devoted hours and hours to her amuseme"
------ temperature: 0.2
ond
of me; and i have devoted hours and hours to her amusement to the consequence to the person the farth of the park of the project gutenberg-tm the least of the proved to the particulars of the person the person any of the pensents of the
------ temperature: 0.5
ticulars of the person the person any of the pensents of the pendeed the spect of the parting so
m

In [28]:
start_index = random.randint(0, len(text) - maxlen - 1)
generated_text = text[start_index: start_index + maxlen]
print('--- Generating with seed: "' + generated_text + '"')
    
sys.stdout.write(generated_text)
temperature = 0.35

# We generate 1800 characters
for i in range(1800):
    sampled = np.zeros((1, maxlen, len(chars)))
    for t, char in enumerate(generated_text):
        sampled[0, t, char_indices[char]] = 1.

    preds = model.predict(sampled, verbose=0)[0]
    next_index = sample(preds, temperature)
    next_char = chars[next_index]

    generated_text += next_char
    generated_text = generated_text[1:]

    sys.stdout.write(next_char)
    sys.stdout.flush()
print()

--- Generating with seed: "istened, wondered, doubted, and was impatient for more. mrs."
istened, wondered, doubted, and was impatient for more. mrs. bennet was the c

/home/ubuntu/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:3: RuntimeWarning: divide by zero encountered in log
  app.launch_new_instance()


ompohed to her sister the lively accusait to him without to him to speak, and the compared to expect to be of the country. but anything of what her country. elizabet in the mentions with his probablery to be accountroficed to him of the party; and the time and sirchishal to staired to the sure i should be as they were not the counted the project gutenberg which the wisted the same to her easily with the counted the sable of him to speed to be dine of the discourtry to decersed to his proper of the rest of the words of the acquaintance of the country. but i should be any discourtress, and who have been the project gutenberg the compared to his sisters in the country. but he added to speak the same
of his sister of her formation of her own a project gutenberg-tm the project gutenberg that he was so any provinity in the room, she was been to comprehend to the project gutenberg the same of the present of a most quite a very country. my dear of her sister of being with the more interrunt as

In [29]:
temperature = 0.45

# We generate 1800 characters
for i in range(1800):
    sampled = np.zeros((1, maxlen, len(chars)))
    for t, char in enumerate(generated_text):
        sampled[0, t, char_indices[char]] = 1.

    preds = model.predict(sampled, verbose=0)[0]
    next_index = sample(preds, temperature)
    next_char = chars[next_index]

    generated_text += next_char
    generated_text = generated_text[1:]

    sys.stdout.write(next_char)
    sys.stdout.flush()
print()

 was not be any of my expect to
seevery witho

/home/ubuntu/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:3: RuntimeWarning: divide by zero encountered in log
  app.launch_new_instance()


ut mentionions of the country of his probers was as the rest them to scarcely meryton, and for the cousting him with a the former to she could be surce to the
constriation of her sister of my fidd everybody wished of my increased whether, and her proved to the family of the family for
the room.

“i should be sure anyone to scided to him to her formation of the streptred any project gutenberg-tm the same of we comproprety without mentioned her at as the meand to her former to she had been foressely menty of his eyes of his father was proved to be not to staying, in the house, and the dear as therefore could be so feelings when she was so much the destrowing to anything to the project gutenberg to the party; and it was about the strept them. and they could not attemption of they spoke. it which i shall contrady and the perfection to her easious and expecting the brother. i should be so supposion of his forminy of the compose the money in the country. but i shall not her former was so set

In [30]:
temperature = 0.75

# We generate 1800 characters
for i in range(1800):
    sampled = np.zeros((1, maxlen, len(chars)))
    for t, char in enumerate(generated_text):
        sampled[0, t, char_indices[char]] = 1.

    preds = model.predict(sampled, verbose=0)[0]
    next_index = sample(preds, temperature)
    next_char = chars[next_index]

    generated_text += next_char
    generated_text = generated_text[1:]

    sys.stdout.write(next_char)
    sys.stdout.flush()
print()

ith the door unit

/home/ubuntu/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:3: RuntimeWarning: divide by zero encountered in log
  app.launch_new_instance()


ed. there must consequent grounds of the idearion in my hume which herself. as she had subject was not to him. lodys his thank as they could be to his condented to here my long
of the famity of the ugaring to him of refund as
the lady fortune of his profect and complace licently believed make the sent for my formed at thesearss from her
heart, and
enuation she was herself. i never from be bow they different him to her former for the room as should be with mind probers for the project weaknescest your being and the distroun could not not be imprudy, soon so of a greate of such a scompanious attempt to added to be a rejoice your sister. but mrs. bennet for the commind except an expressions, in the surpried the whose everybody make negard.

“i would not hope itsome,
she suffers to her to scilinal
decersed of the more any
as she would not have could at shall of neithered were a willinial to her the sext sime in
them, and she would not accompaty, and as to what they were myrefing than on wh